# Discovery

By now, world population is in constant increase. The more the people, the more the food we need that translate in an increase of use of natural resources.


But which are the product we produce that exploit most resources? Which countries contribute to the exploitation of natural resources?

**Goal**: Discovering which are the top 10 products that use more water, need more land and emit more gas. Then discovering which countries are the most producer of each category


# Data Selection

In [201]:
import pandas as pd
import numpy as np
import sidetable as stb

In [202]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 100
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [203]:
fao_df = pd.read_csv('FAO.csv', encoding='latin-1')
food_pr_df = pd.read_csv('Food_Production.csv')

# Data Cleaning

### FAO Data

In [204]:
fao_df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,"1,928.00","1,904.00","1,666.00","1,950.00","2,001.00","1,808.00","2,053.00","2,045.00","2,154.00","1,819.00","1,963.00","2,215.00","2,310.00","2,335.00","2,434.00","2,512.00","2,282.00","2,454.00","2,443.00","2,129.00","2,133.00","2,068.00","1,994.00","1,851.00","1,791.00","1,683.00","2,194.00","1,801.00","1,754.00","1,640.00","1,539.00","1,582.00","1,840.00","1,855.00","1,853.00","2,177.00","2,343.00","2,407.00","2,463.00","2,600.00","2,668.00","2,776.00","3,095.00","3,249.00","3,486.00","3,704.00","4,164.00","4,252.00","4,538.00","4,605.00","4,711.00",4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.00,183.00,182.00,220.00,220.00,195.00,231.00,235.00,238.00,213.00,205.00,233.00,246.00,246.00,255.00,263.00,235.00,254.00,270.00,259.00,248.00,217.00,217.00,197.00,186.00,200.00,193.00,202.00,191.00,199.00,197.00,249.00,218.00,260.00,319.00,254.00,326.00,347.00,270.00,372.00,411.00,448.00,460.00,419.00,445.00,546.00,455.00,490.00,415.00,442.00,476.00,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.00,76.00,76.00,76.00,76.00,75.00,71.00,72.00,73.00,74.00,71.00,70.00,72.00,76.00,77.00,80.00,60.00,65.00,64.00,64.00,60.00,55.00,53.00,51.00,48.00,46.00,46.00,47.00,46.00,43.00,43.00,40.00,50.00,46.00,41.00,44.00,50.00,48.00,43.00,26.00,29.00,70.00,48.00,58.00,236.00,262.00,263.00,230.00,379.00,315.00,203.00,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.00,237.00,237.00,238.00,238.00,237.00,225.00,227.00,230.00,234.00,223.00,219.00,225.00,240.00,244.00,255.00,185.00,203.00,198.00,202.00,189.00,174.00,167.00,160.00,151.00,145.00,145.00,148.00,145.00,135.00,132.00,120.00,155.00,143.00,125.00,138.00,159.00,154.00,141.00,84.00,83.00,122.00,144.00,185.00,43.00,44.00,48.00,62.00,55.00,60.00,72.00,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.00,210.00,214.00,216.00,216.00,216.00,235.00,232.00,236.00,200.00,201.00,216.00,228.00,231.00,234.00,240.00,228.00,234.00,228.00,226.00,210.00,199.00,192.00,182.00,173.00,170.00,154.00,148.00,137.00,144.00,126.00,90.00,141.00,150.00,159.00,108.00,90.00,99.00,72.00,35.00,48.00,89.00,63.00,120.00,208.00,233.00,249.00,247.00,195.00,178.00,191.00,200,200


In [205]:
fao_df.shape

(21477, 63)

#### Checking columns dtypes

In [206]:
# for col in fao_df.columns:
#     print(f'{col :<25}{fao_df[col].dtypes}', end='\n\n')
types = [fao_df[col].dtypes for col in fao_df.columns]

dtype_df = pd.DataFrame(data = types, index=fao_df.columns, columns=['DTypes'] )
dtype_df.T

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
DTypes,object,int64,object,int64,object,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64


changing *Y2012* and *Y2013* data type from int to float

In [207]:
fao_df[['Y2012', 'Y2013']] = fao_df[['Y2012', 'Y2013']].astype(float)

#### Checking column names and eventually fixing them

In [208]:
fao_df.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013'],
      dtype='object')

In [209]:
fao_df.rename(columns={n:n[1:] for n in fao_df.columns if 'Y' in n}, inplace=True) 

#### Checking for duplicates

In [210]:
print(f'Number of rows before dropping duplicates: {fao_df.shape[0] :>6}')
fao_df.drop_duplicates(keep='first')
print(f'Number of rows after dropping duplicates: {fao_df.shape[0] :>7}')

Number of rows before dropping duplicates:  21477
Number of rows after dropping duplicates:   21477


Counting the number of rows before checking for duplicates, and after, i can see there are no duplicates


#### Checking qualitative columns values

In [211]:
years = fao_df.columns[fao_df.columns.get_loc('1961'):]
fao_df.stb.counts(exclude=['number'])

,count,unique,most_freq,most_freq_count,least_freq,least_freq_count
Unit,21477,1,1000 tonnes,21477,1000 tonnes,21477
Element,21477,2,Food,17528,Feed,3949
Item,21477,115,Milk - Excluding Butter,558,"Meat, Aquatic Mammals",3
Area Abbreviation,21477,169,CHN,541,LSO,75
Area,21477,174,Spain,150,Lesotho,75


In [212]:
to_check = ['Area', 'Area Abbreviation', 'Item']
for col in to_check:
    print(f'{col}\n{sorted(fao_df[col].unique())}', end=f'\n{"-"*100}\n')

Area
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China, Hong Kong SAR', 'China, Macao SAR', 'China, Taiwan Province of', 'China, mainland', 'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', "Côte d'Ivoire", "Democratic People's Republic of Korea", 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Icel

 *Area Abbreviation* and the *Area* count do not match. Let's investigate why

In [213]:
# checking if there are more than 1 country with the same area code
a_code_unique = fao_df.groupby('Area Abbreviation')['Area'].nunique()
a_code_unique[a_code_unique > 1]

Area Abbreviation
AZE    2
CHN    4
THA    2
Name: Area, dtype: int64

As suspected, 3 *Area Abbreviation* are associated with more than 1 country. Let's find out which countries fall under the same code

In [214]:
cond = fao_df['Area Abbreviation'].isin(['AZE', 'THA', 'CHN'])
fao_df.loc[cond].groupby('Area Abbreviation')['Area'].unique()

Area Abbreviation
AZE                                                                   [Azerbaijan, Bahamas]
CHN    [China, Hong Kong SAR, China, Macao SAR, China, mainland, China, Taiwan Province of]
THA                                   [Thailand, The former Yugoslav Republic of Macedonia]
Name: Area, dtype: object

The result of the investigation evidenciate that *Bahamas* and *The former Yugoslav Republic of Macedonia* fall under the wrong *Area abbreviation* code.
Since we don't need that column, we can stop here and just drop it


#### Dropping unecessary columns

In [215]:
fao_df.drop(columns=['Area Abbreviation', 'Area Code', 'Item Code', 'Element Code', 'Unit'], inplace=True)

#### Fixing Area column values

After a close look, some of the Area names are not correct. let's correct them:

In [216]:
Rename_area = {'Bolivia (Plurinational State of)':'Bolivia',
                      'Cabo Verde':'Cape Verde',
                      'China, mainland':'China',
                      'China, Hong Kong SAR':'Hong Kong',
                      'China, Macao SAR':'Macao',
                      'China, Taiwan Province of':'Taiwan',
                      'Czechia':'Czech Republic',
                      "Democratic People's Republic of Korea":'North Korea',
                      'Iran (Islamic Republic of)':'Iran',
                      'Republic of Korea':'South Korea',
                      'The former Yugoslav Republic of Macedonia':'Macedonia',
                      'Venezuela (Bolivarian Republic of)':'Venezuela',
                      }

fao_df['Area'].rename(Rename_area , inplace=True)

#### Checking latitude and longitude values

Let's check if latitude and longitude contains some non valide values. Latitude ranges from -90 to 90,  longitude ranges from -180 to 180

In [217]:
fao_df[['longitude', 'latitude']].agg([min, max])

,longitude,latitude
min,-172.10,-40.90
max,179.41,64.96


Both max and min of latitude and longitude fall under the accepted range

#### Checking if there are some invalid values in the *Years* columns

Let's check if are there any negative numbers in production

In [218]:
cond = fao_df.loc[:,'1961':'2013'].agg([min])
cond.T[cond.T['min'] < 0]

,min
2012,-169.00
2013,-246.00


Now let's investigate which country has a negative amount of production for which *Item* and *Element* (food or feed)

In [219]:
fao_df.loc[fao_df[['2013', '2012']].idxmin().unique(), ['Area', 'Item', 'Element']]

,Area,Item,Element
10082,Japan,Oats,Food


negative number in production must be an error of input, just drop the entire row

In [220]:
fao_df.drop(labels=10082, inplace=True)

#### Checking for missing values

In [221]:
fao_df.stb.missing(clip_0=True, style=True)

,missing,total,percent
1961,"3,539","21,476",16.48%
1976,"3,539","21,476",16.48%
1989,"3,539","21,476",16.48%
1988,"3,539","21,476",16.48%
1987,"3,539","21,476",16.48%
1962,"3,539","21,476",16.48%
1985,"3,539","21,476",16.48%
1984,"3,539","21,476",16.48%
1983,"3,539","21,476",16.48%
1982,"3,539","21,476",16.48%


Let's create a dataframe containing only the years variables

In [222]:
year_df = fao_df[years]

First let's drop all the rows that contain all NaN values, if there are any

In [223]:
fao_df = fao_df.dropna(how='all')

Since the only NaN values in the dataframe are in the numeric years data set, let's fill all the remaining NaN values with 0

In [224]:
fao_df.fillna(0, inplace=True)

Lastly we  create a list of index of the rows of the years dataframe that has only 0 values

In [225]:
idx_to_drop = year_df.loc[(year_df == 0).all(axis=1)].index

Finally droping those rows

In [226]:
fao_df = fao_df.drop(labels=idx_to_drop)

# Data Exploration/Data Transformation

# Data Visualization

In [227]:
# cond = fao_df['Element'] == 'Feed'
# cond2 = fao_df['Element'] == 'Food'

# feed = fao_df.loc[cond, 'Item']
# food = fao_df.loc[cond2, 'Item']

# differences = np.setdiff1d(food, feed)
# for n, i in enumerate(differences):
#     print(f'{n}{i :>30}')